In [1]:
import numpy as np

In [30]:
n = 5 ## grid size

# STATES
S = []
for row in range(0,n):
    for col in range(0,n):
        S.append((col,row))
print("States")
print(S)
print("Number of states: ", len(S))
print("\n")

# ACTIONS
A = [0, 1, 2, 3, 4]
print("Actions")
print(A)
print("Number of actions: ", len(A))
print("\n")

# TRANSITION PROBABILITIES
P = np.zeros((n**2, len(A), n**2))

print("Transition Probabilities")
print("Shape of the transition probability matrix: ", P.shape)
print("Number of transition probabilities: ", P.shape[0]* P.shape[1]* P.shape[2])



States
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (0, 3), (1, 3), (2, 3), (3, 3), (4, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Number of states:  25


Actions
[0, 1, 2, 3, 4]
Number of actions:  5


Transition Probabilities
Shape of the transition probability matrix:  (25, 5, 25)
Number of transition probabilities:  3125
